# LangChain - Quickstart Guide - The Basics - Part 2 (ChatModel)

### LangChain is a framework which simplifies the development of apps using LLMs like OpenAI's GPT-4

##### Quickstart Guide location = https://python.langchain.com/en/latest/getting_started/getting_started.html

### Install LangChain and OpenAI

In [ ]:
pip install langChain

In [ ]:
pip install OpenAI

In [ ]:
pip install google-search-results

#### Lets Import OS and Set the API Keys

We need to set the environment variable "OPENAI_API_KEY" and SERPAPI_API_KEY

In [1]:
import os

In [2]:
os.environ["OPENAI_API_KEY"] = ""

In [3]:
os.environ["SERPAPI_API_KEY"] = ""

#### Import the ChatOpenAI wrapper

In [4]:
#from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


### Chat Models:  Use LLM under the hood. 

The interface is different from what we saw with LLM. Instead of text in text out Chat Models are message in message out as inputs and outputs.

#### Let's also import the schemas for: AIMessage, HumanMessage and  SystemMessage

In [5]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

#### Create the chat model

In [6]:
chat = ChatOpenAI(temperature = 0)

#### Get message completion from the chat model by passing in a single message

In [7]:
chat([HumanMessage(content = "Translate this sentence from English to French. It is a beautiful day!")])

AIMessage(content="C'est une belle journée !", additional_kwargs={}, example=False)

#### Pass in Multiple messages for OpenAI’s GPT-3.5-turbo and GPT-4 models.

In [8]:
chat = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo')
messages = [
    SystemMessage(content = "You are a helpful assistant that translates English to French"),
    HumanMessage(content="It is a beautiful day!")
]

chat(messages)


AIMessage(content="C'est une belle journée !", additional_kwargs={}, example=False)

#### Construct multiple completions by passing multiple sets of messages and calling 'generate' on the chat to return LLMResult


In [9]:
batch_messages = [
    [
        SystemMessage(content = "You are a helpful assistant that translates English to French"),
        HumanMessage(content = "It is a beautiful day!")
    ],
    [
        SystemMessage(content = "You are a helpful assistant that translates English to French"),
        HumanMessage(content = "Let's go for a hike!")
    ],
]

result = chat.generate(batch_messages)
result


LLMResult(generations=[[ChatGeneration(text="C'est une belle journée !", generation_info=None, message=AIMessage(content="C'est une belle journée !", additional_kwargs={}, example=False))], [ChatGeneration(text='Allons faire une randonnée !', generation_info=None, message=AIMessage(content='Allons faire une randonnée !', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 59, 'completion_tokens': 14, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo'})

#### LLMResult can be queried for things like token usage

In [10]:
result.llm_output['token_usage']


{'prompt_tokens': 59, 'completion_tokens': 14, 'total_tokens': 73}

### ChatPromtTemplates


In [11]:
# load the templates
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# create the chat

chat = ChatOpenAI(temperature = 0)

# set the system message prompt template

system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# set the human message prompt template

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# creaet the chat prompt template

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# format the prompt using the chat template and input for the templates. Then generate out put.

chat(chat_prompt.format_prompt(input_language="English", 
                               output_language = "French", 
                               text = "It is a beautiful day!").to_messages())


AIMessage(content="Il fait beau aujourd'hui !", additional_kwargs={}, example=False)

### Chains - Combine Chat Models and ChatPromptTemplates into a multi-step workflow

In [12]:
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    )

chat = ChatOpenAI(temperature = 0)

# set the system message prompt template

system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# set the human message prompt template

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Set the chat prompt template using the system and human prompt template

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


In [13]:
# create and run the chain using the chat prompt template we just set

chain = LLMChain(llm = chat, prompt = chat_prompt)
chain.run(input_language = "English", output_language = "French", text = "It is a beautiful day!")


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a3f08db4008a99764362e29e57db00fd in your message.).


"Il fait beau aujourd'hui !"

### Agents - Dynamically call Chains based on user inputs

Use agents to determine which action to take and in what order. The use is similar for chat models to the llm.

A tool is a function that performs a specified function. This can be things like: Google Search, Database lookup, Python REPL, other chains. 

Agents: For a list of supported agents and their specifications, https://python.langchain.com/en/latest/modules/agents/agents.html

Tools: For a list of predefined tools and their specifications, https://python.langchain.com/en/latest/modules/agents/tools.html.



In [17]:
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# Let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Now let's load tools to use we plan to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the chat model, and the type of agent we want to use.

agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("How tall is Niagara Falls? What would be its height if it was two times as tall?")



> Entering new AgentExecutor chain...
Question: How tall is Niagara Falls? What would be its height if it was two times as tall?
Thought: I need to use a search engine to find the height of Niagara Falls and then use a calculator to double that height.
Action:
```
{
  "action": "Search",
  "action_input": "Niagara Falls height"
}
```

Observation: 325′
Thought:Now I need to use a calculator to double the height of Niagara Falls.
Action:
```
{
  "action": "Calculator",
  "action_input": "325*2"
}
```


Observation: Answer: 650
Thought:I now know the final answer.
Final Answer: Niagara Falls is 325 feet tall. If it was two times as tall, it would be 650 feet tall.

> Finished chain.


'Niagara Falls is 325 feet tall. If it was two times as tall, it would be 650 feet tall.'

### Memory - Add State to Chains and Agents

If our application requires our chains and agent to have memory of the previous conversation we use ConversationChain.
The difference between using ConversationChain with llm vs chat)_modl is that, the messages can be kept in their own memory object instead of being condensed to a string. 


In [18]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
            MessagesPlaceholder(variable_name = "history"),
            HumanMessagePromptTemplate.from_template("{input}")
        ])



In [19]:
# set up the chat and conversation chain

chat = ChatOpenAI(temperature = 0)
memory = ConversationBufferMemory(return_messages = True)
conversation = ConversationChain(memory = memory, prompt = prompt, llm = chat)


In [20]:
# predict the next response

conversation.predict(input="Hello, how are you?")

"Hello! I'm an AI language model, so I don't have feelings like humans do, but I'm functioning properly and ready to assist you. How can I help you today?"

In [22]:
# Continue the conversation to the next predict

conversation.predict(input="It is a beautiful day! Going out for a hike would be nice. Good hikes in the Catskills?")

"Yes, the Catskills are a great place for hiking! There are many trails to choose from, ranging from easy to difficult. Some popular hikes in the Catskills include Kaaterskill Falls, Overlook Mountain, and Devil's Path. Kaaterskill Falls is a two-stage waterfall that is one of the most popular hikes in the area. Overlook Mountain is a moderate hike that offers stunning views of the Hudson Valley. Devil's Path is a challenging hike that is known for its steep climbs and rocky terrain."

In [23]:
# Continue the conversation to the next predict

conversation.predict(input="Thanks! Tell me about yourself ")


"Sure! As I mentioned earlier, I'm an AI language model designed to assist with various tasks such as answering questions, generating text, and providing information. I was created using advanced machine learning algorithms and trained on a large dataset of text. My goal is to provide accurate and helpful responses to your queries. I don't have personal preferences or emotions like humans do, but I'm always here to help you with any questions or tasks you may have."

In [24]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, how are you?', additional_kwargs={}, example=False), AIMessage(content="Hello! I'm an AI language model, so I don't have feelings like humans do, but I'm functioning properly and ready to assist you. How can I help you today?", additional_kwargs={}, example=False), HumanMessage(content='It is a beautiful day! Going out for a hike would be nice. Good hikes in the Catskills?', additional_kwargs={}, example=False), AIMessage(content="Yes, the Catskills are a great place for hiking! There are many trails to choose from, ranging from easy to difficult. Some popular hikes in the Catskills include Kaaterskill Falls, Overlook Mountain, and Devil's Path. Kaaterskill Falls is a two-stage waterfall that is one of the most popular hikes in the area. Overlook Mountain is a moderate hike that offers stunning views of the Hudson Valley. Devil's Path is a challenging hike that is known for its steep climbs a

In [25]:
history

import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""
#from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
chat = ChatOpenAI(temperature = 0)
chat([HumanMessage(content = "Translate this sentence from English to French. It is a beautiful day!")])
chat = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo')
messages = [
    SystemMessage(content = "You are a helpful assistant that translates English to French"),
    HumanMessage(content="It is a beautiful day!")
]

chat(messages)
batch_messages = [
    [
        SystemMessage(content = "You are a helpful assistant that translates English to French"),
        HumanMessage(content = "It is a beautiful day!")
    ],
    [
        SystemMessage(content = "You are a helpful assistant that translates English to French"),
        HumanMessage(content = "Let's go for a hike!")
    ],
]

result = chat.genera